In [8]:
import numpy as np

import pandas as pd

%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import (
    KFold, StratifiedKFold, train_test_split)
from sklearn.feature_selection import RFECV

from sklearn.ensemble import RandomForestClassifier

import joblib

from libs.container import Container
import dataset

In [2]:
cpu = joblib.cpu_count()

In [3]:
data = dataset.load_scaled()

df = pd.concat([data.b278, data.b261])

cls = {name: idx for idx, name in enumerate(df.tile.unique())}
df["cls"] = df.tile.apply(cls.get)

print(cls)

del data

Reading '/mnt/is0/jbcabral/denoise/dataset/full_scaled.pkl.bz2'
{'b278': 0, 'b261': 1}


In [9]:
BEST_PARAMS = joblib.load("results/hp_selection.pkl.bz2")
RF_PARAMS = BEST_PARAMS["rf"].best_params_

del BEST_PARAMS

RF_PARAMS

{'criterion': 'entropy',
 'max_features': 'sqrt',
 'min_samples_split': 5,
 'n_estimators': 500,
 'n_jobs': 10}

In [7]:
X = df[dataset.FEATURES].values
y = df.cls.values

In [10]:
%%time
clf = RandomForestClassifier(**RF_PARAMS)
sel = RFECV(clf, n_jobs=-1, cv=10)
sel.fit(X, y)

CPU times: user 31min 14s, sys: 8.9 s, total: 31min 23s
Wall time: 12min 36s


RFECV(cv=10,
      estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                       class_weight=None, criterion='entropy',
                                       max_depth=None, max_features='sqrt',
                                       max_leaf_nodes=None, max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=5,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=500, n_jobs=10,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False),
      min_features_to_select=1, n_jobs=-1, scoring=None, step=1, verbose=0)

In [11]:
joblib.dump(sel, "results/rfecv.pkl.bz2", compress=3)

['results/rfecv.pkl.bz2']